In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

In [3]:
p = Path('/home/khali/scams/Results')


In [10]:
LGBM_p = []
for it in p.iterdir():
    if str(it).split('/')[-1].split('_')[0] == 'LGBM':
        LGBM_p.append(str(it))
        

In [19]:
a = pd.read_csv(Path(LGBM_p[0]) / 't-test_stats.csv', index_col=0)

In [74]:
Metrics = a['Metrics'].to_list()

In [67]:
def gather_stats(df_stats, metric):
    result = []
    for it in metric:
        AL = df_stats[df_stats['Metrics'] == it]['Mean AL'].to_list()[0]
        non_AL = df_stats[df_stats['Metrics'] == it]['Mean non AL'].to_list()[0]
        log = df_stats[df_stats['Metrics'] == it]['is_significant'].to_list()[0]
        if AL > non_AL and log:
            result.append('AL_T')
        if AL > non_AL and not log:
            result.append('AL_F')
        if AL < non_AL and log:
            result.append('N_AL_T')
        if AL < non_AL and not log:
            result.append('N_AL_F')
        if AL == non_AL:
            result.append('E')
    return result


In [68]:
df_summry_stat = {}
for p in LGBM_p:
    c_p = Path(p) / 't-test_stats.csv'
    data_s = pd.read_csv(c_p, index_col=0)
    iter_res = gather_stats(data_s, Metrics)
    df_summry_stat[p.split('/')[-1]] = iter_res
    

In [72]:
df_summry_stat = pd.DataFrame.from_dict(df_summry_stat, orient='index', columns=Metrics)

In [73]:
df_summry_stat.to_csv('/home/khali/scams/Results/LGBM_summary.csv')

In [77]:
values_counts = df_summry_stat.apply(pd.value_counts)


In [78]:
values_counts.to_csv('/home/khali/scams/Results/LGBM_summary_values_counts.csv')

In [86]:
values_counts.iloc[:, 0:7].fillna(0)

,AUC_LB_test,AUC_test,AUC_UB_test,Accuracy_test,F1_test,MCC_test,AUC_LB_validation
AL_F,16.0,15.0,15.0,4.0,13.0,5.0,6.0
AL_T,17.0,18.0,18.0,29.0,20.0,27.0,27.0
N_AL_F,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N_AL_T,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [87]:
values_counts.iloc[:, 7:].fillna(0)

,AUC_validation,AUC_UB_validation,Accuracy_validation,F1_validation,MCC_validation
AL_F,6.0,6.0,2,2,3.0
AL_T,27.0,27.0,28,25,29.0
N_AL_F,0.0,0.0,2,4,0.0
N_AL_T,0.0,0.0,1,2,1.0
